# Organelle Morpholgy analysis

In the following we will demonstrate the `organelle_morphology` package.

In [ ]:
import organelle_morphology

In [ ]:
project_path = "../data/cebra_em_example/seg_er_5nm_mito_10nm/"

In [ ]:
p = organelle_morphology.Project(project_path)

In [ ]:
p.available_sources()

In [ ]:
p.add_source("mito-it00_b0_6_stitched", organelle="mito")
p.compression_level = 2

In [ ]:
geo = p.geometric_properties
geo.head(5)

In [ ]:
# filter only water tight meshes
ids = geo[geo["water_tight"] == True].index.to_list()

In [ ]:
p.show(ids=ids, show_morphology=True, height=1000)

In [ ]:
org = p.organelles("mito_0099")[0]
org

In [ ]:
import skeletor as sk

In [ ]:
fixed_mesh = sk.pre.fix_mesh(org.mesh)
# cont = sk.pre.contract(fixed_mesh, epsilon=0.05)
# skel = sk.skeletonize.by_vertex_clusters(cont, sampling_dist=6)
skel = sk.skeletonize.by_wavefront(fixed_mesh, step_size=10, waves=2)

skel.mesh = fixed_mesh
sk.post.radii(skel, method="knn")
sk.post.clean_up(skel, inplace=True)
skel

In [ ]:
import plotly.graph_objects as go

# Assuming nodes is a list of tuples (x, y) and edges is a list of tuples (node1, node2)
nodes = skel.vertices
edges = skel.edges
line_width = 2
import plotly.graph_objects as go

# Assuming nodes is a list of tuples (x, y) and edges is a list of tuples (node1, node2)
nodes = skel.vertices
edges = skel.edges

line_width = 10

# Create a 3D line plot for the edges
x_values = []
y_values = []
z_values = []

for edge in edges:
    x_values.extend(
        [nodes[edge[0]][0], nodes[edge[1]][0], None]
    )  # add None to separate lines
    y_values.extend(
        [nodes[edge[0]][1], nodes[edge[1]][1], None]
    )  # add None to separate lines
    z_values.extend(
        [nodes[edge[0]][2], nodes[edge[1]][2], None]
    )  # add None to separate lines

edge_trace = go.Scatter3d(
    x=x_values,
    y=y_values,
    z=z_values,
    mode="lines",
    line=dict(width=line_width),  # Set line width
    name="Edge",  # Set label
)

In [ ]:
p.show(ids="mito_0099", show_morphology=False, additional_object=edge_trace)

In [ ]:
skel.radius

In [ ]:
scene = skel.scene()

In [ ]:
test = list(scene.geometry.values())[0]

In [ ]:
test.entities

In [ ]:
import trimesh

test2 = trimesh.path.traversal.resample_path(test.vertices, step=0.5)